Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/training-with-deep-learning/how-to-use-estimator/how-to-use-estimator.png)

# How to use Estimator in Azure ML

## Introduction
This tutorial shows how to use the Estimator pattern in Azure Machine Learning SDK. Estimator is a convenient object in Azure Machine Learning that wraps run configuration information to help simplify the tasks of specifying how a script is executed.


## Prerequisite:
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning
* If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [configuration notebook](../../../configuration.ipynb) to:
    * install the AML SDK
    * create a workspace and its configuration file (`config.json`)

Let's get started. First let's import some Python libraries.

In [1]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.41


## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [2]:
ws = Workspace.from_config()
# print('Workspace name: ' + ws.name, 
#       'Azure region: ' + ws.location, 
#       'Subscription id: ' + ws.subscription_id, 
#       'Resource group: ' + ws.resource_group, sep = '\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


## Create an Azure ML experiment
Let's create an experiment named "estimator-test". The script runs will be recorded under this experiment in Azure.

In [3]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='torchvision')

## Use a train.py script


In [4]:
with open('./train.py', 'r') as f:
    print(f.read())


import sys 
import os
import argparse
sys.path.append('./cocoapi/PythonAPI/')

from PIL import Image
import torch
import xml.etree.ElementTree as ET

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.rpn import RPNHead

import transforms as T
from engine import train_one_epoch, evaluate
import utils

class BuildDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files
        self.imgs = list(sorted(os.listdir(os.path.join(root, 'JPEGImages'))))
        
    def __getitem__(self, idx):
        img_path = os.path.join(self.root, 'JPEGImages', self.imgs[idx])
        xml_path = os.path.join(self.root, 'Annotations', '{}.xml'.format(self.imgs[idx].strip('.jpg')))
        img = Image.open(img_path).convert("RGB")
        
        # parse XML

## Create A Pytorch Estimator

First we import the Pytorch class and also a widget to visualize a run.

In [5]:
from azureml.train.dnn import PyTorch

script_params = {
    '--data_path': '/datadrive/torchvisionOD/BuildData/',
    '--workers': 8,
    '--learning_rate' : 0.005,
    '--epochs' : 5,
    '--anchor_sizes' : '16,32,64,128,256,512',
    '--anchor_aspect_ratios' : '0.25,0.5,1.0,2.0',
    '--rpn_nms_thresh' : 0.5,
    '--box_nms_thresh' : 0.3,
    '--box_score_thresh' : 0.10    
}

estimator = PyTorch(source_directory='.', 
                    script_params=script_params,
                    compute_target='local',
                    entry_script='train.py',
                    use_docker=False,
                    user_managed=True,
                    use_gpu=True)

In [6]:
estimator.run_config.environment.python.interpreter_path = '/data/anaconda/envs/torchdetectaml/bin/python'
estimator.run_config.history.snapshot_project = False
# estimator.run_config.environment.environment_variables["PYTHONPATH"] = "$PYTHONPATH:/datadrive/cocoapi/PythonAPI/"

In [7]:
from azureml.widgets import RunDetails

The simplest estimator is to submit the current folder to the local computer. Estimator by default will attempt to use Docker-based execution. Let's turn that off for now. It then builds a conda environment locally, installs Azure ML SDK in it, and runs your script.

In [8]:
run = exp.submit(estimator)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
run.wait_for_completion(show_output=True)

RunId: torchvision_1560532657_a125710f
Web View: https://mlworkspace.azure.ai/portal/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fboyluamlrg/providers/Microsoft.MachineLearningServices/workspaces/fboyluamlws/experiments/torchvision/runs/torchvision_1560532657_a125710f

Streaming azureml-logs/70_driver_log.txt

Epoch: [0]  [  0/424]  eta: 0:20:10  lr: 0.000017  loss: 2.1649 (2.1649)  loss_classifier: 1.3078 (1.3078)  loss_box_reg: 0.0034 (0.0034)  loss_objectness: 0.6881 (0.6881)  loss_rpn_box_reg: 0.1655 (0.1655)  time: 2.8542  data: 0.2664  max mem: 3722
Epoch: [0]  [ 10/424]  eta: 0:13:57  lr: 0.000135  loss: 1.9251 (1.8368)  loss_classifier: 1.0036 (0.9059)  loss_box_reg: 0.0087 (0.0082)  loss_objectness: 0.6887 (0.6900)  loss_rpn_box_reg: 0.2646 (0.2328)  time: 2.0226  data: 0.0319  max mem: 5977
Epoch: [0]  [ 20/424]  eta: 0:13:30  lr: 0.000253  loss: 1.2237 (1.4852)  loss_classifier: 0.1816 (0.5322)  loss_box_reg: 0.0095 (0.0109)  loss_objectness: 0.6887 (0.

In [10]:
run.get_file_names()

['azureml-logs/60_control_log.txt',
 'azureml-logs/70_driver_log.txt',
 'logs/azureml/azureml.log',
 'outputs/model_latest.pth']